# Data Prep

1. Subsampling to reduce imbalance
2. Handle bert wordpiece tokens and align it with provided tokens (50.94 etc)

In [1]:
import pandas as pd
import datasets

In [2]:
pd.options.display.max_colwidth = 1000

In [3]:
selected_tag_ids = [34, 37, 38, 41, 42, 43, 87, 178, 199, 204, 0]

In [4]:
dataset = datasets.load_dataset("nlpaueb/finer-139")

In [5]:
ner_tags = dataset["train"].features["ner_tags"].feature.names

ner_tags

['O',
 'B-AccrualForEnvironmentalLossContingencies',
 'B-AcquiredFiniteLivedIntangibleAssetsWeightedAverageUsefulLife',
 'I-AcquiredFiniteLivedIntangibleAssetsWeightedAverageUsefulLife',
 'B-AllocatedShareBasedCompensationExpense',
 'B-AmortizationOfFinancingCosts',
 'B-AmortizationOfIntangibleAssets',
 'I-AmortizationOfIntangibleAssets',
 'B-AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount',
 'I-AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount',
 'B-AreaOfRealEstateProperty',
 'I-AreaOfRealEstateProperty',
 'B-AssetImpairmentCharges',
 'B-BusinessAcquisitionEquityInterestsIssuedOrIssuableNumberOfSharesIssued',
 'B-BusinessAcquisitionPercentageOfVotingInterestsAcquired',
 'I-BusinessAcquisitionPercentageOfVotingInterestsAcquired',
 'B-BusinessCombinationAcquisitionRelatedCosts',
 'B-BusinessCombinationConsiderationTransferred1',
 'B-BusinessCombinationContingentConsiderationLiability',
 'B-BusinessCombinationRecognizedIdentifiableAssetsAcquired

In [6]:
[ner_tags[x] for x in selected_tag_ids]

['B-DebtInstrumentBasisSpreadOnVariableRate1',
 'B-DebtInstrumentFaceAmount',
 'I-DebtInstrumentFaceAmount',
 'B-DebtInstrumentInterestRateStatedPercentage',
 'B-DebtInstrumentMaturityDate',
 'I-DebtInstrumentMaturityDate',
 'B-LineOfCreditFacilityMaximumBorrowingCapacity',
 'I-DebtInstrumentInterestRateStatedPercentage',
 'I-LineOfCreditFacilityMaximumBorrowingCapacity',
 'I-DebtInstrumentBasisSpreadOnVariableRate1',
 'O']

In [7]:
# test_df = pd.DataFrame.from_dict(dataset["test"])

train_df = pd.DataFrame.from_dict(dataset["train"])

valid_df = pd.DataFrame.from_dict(dataset["validation"])

In [8]:

train_df["ner_tags_subset"] = train_df["ner_tags"].apply(lambda x: [y if y in selected_tag_ids else 0 for y in x])

valid_df["ner_tags_subset"] = valid_df["ner_tags"].apply(lambda x: [y if y in selected_tag_ids else 0 for y in x])

train_df.sample((10))

,id,tokens,ner_tags,ner_tags_subset
676374,676374,"[This, complaint, is, purportedly, brought, derivatively, on, behalf, of, Tree, House, and, asserts, state, law, claims, against, certain, officers, for, breach, of, fiduciary, duty, ,, unjust, enrichment, ,, abuse, of, control, ,, gross, mismanagement, ,, and, corporate, waste, ,, in, addition, to, asserting, violations, of, Section, 14, of, the, Securities, Exchange, Act, of, 1934, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
724156,724156,"[Our, ability, to, make, scheduled, payments, of, the, principal, of, ,, to, pay, interest, on, ,, or, to, refinance, our, indebtedness, ,, including, the, amounts, payable, under, the, 2024, Notes, ,, depends, on, our, future, performance, ,, which, is, subject, to, economic, ,, financial, ,, competitive, ,, and, other, factors, beyond, our, control, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
846288,846288,"[The, remaining, shares, will, be, retired, in, 2020, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
793572,793572,"[In, addition, ,, at, any, time, on, or, prior, to, November, 15, ,, 2022, ,, up, to, 40, percent, of, the, aggregate, principal, amount, of, the, 3, 7/8, percent, Notes, may, be, redeemed, with, the, net, cash, proceeds, of, certain, equity, offerings, at, a, redemption, price, equal, to, 103.875, percent, of, the, aggregate, principal, amount, of, the, notes, plus, accrued, and, unpaid, interest, ,, if, any, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 44, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 44, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
84748,84748,"[No, subsequent, events, or, transactions, were, identified, ., On, January, 1, ,, 2018, ,, the, Company, retrospectively, adopted, the, new, accounting, standard, for, revenue, recognition, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
39048,39048,"[The, Company, obtains, one, price, per, security, .]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]"
822435,822435,"[Segment, Information, We, have, three, reportable, operating, segments, :, brokerage, ,, risk, management, and, corporate, .]","[0, 0, 0, 0, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
681544,681544,"[This, policy, was, effective, January, 1, ,, 2019, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
660598,660598,"[On, February, 12, ,, 2018, ,, the, court, granted, in, part, and, denied, in, part, the, Company, ’, s, motion, to, dismiss, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
434772,434772,"[However, ,, certain, of, these, deferred, tax, assets, and, deferred, tax, liabilities, relate, to, income, tax, effects, initially, recognized, at, the, 35, %, tax, rate, through, other, comprehensive, income, (, loss, ), on, items, reported, within, accumulated, other, comprehensive, income, (, loss, ), on, an, entity, ’, s, balance, sheet, .]","[0, 0, 0, 0, 0, 0,

In [9]:
sub_dist = train_df["ner_tags_subset"].explode().value_counts()

sub_dist = sub_dist.reset_index()

sub_dist['ner_tags_subset2'] = sub_dist['ner_tags_subset'].apply(lambda x: ner_tags[x])

sub_dist

,ner_tags_subset,count,ner_tags_subset2
0,0,40772643,O
1,41,15208,B-DebtInstrumentInterestRateStatedPercentage
2,87,11693,B-LineOfCreditFacilityMaximumBorrowingCapacity
3,34,11445,B-DebtInstrumentBasisSpreadOnVariableRate1
4,37,10749,B-DebtInstrumentFaceAmount
5,43,4320,I-DebtInstrumentMaturityDate
6,42,1440,B-DebtInstrumentMaturityDate
7,38,1,I-DebtInstrumentFaceAmount


##### Checking distributions

In [9]:
(train_df["ner_tags"].apply(sum) > 0).value_counts(normalize=True)

ner_tags
False    0.800979
True     0.199021
Name: proportion, dtype: float64

In [8]:
(train_df["ner_tags_subset"].apply(sum) > 0).value_counts(normalize=True)

False    0.967838
True     0.032162
Name: ner_tags_subset, dtype: float64

In [9]:
(valid_df["ner_tags_subset"].apply(sum) > 0).value_counts(normalize=True)

False    0.965972
True     0.034028
Name: ner_tags_subset, dtype: float64

In [10]:
(valid_df["ner_tags"].apply(sum) > 0).value_counts(normalize=True)

False    0.807963
True     0.192037
Name: ner_tags, dtype: float64

In [11]:
train_df["subset_present"] = (train_df["ner_tags_subset"].apply(sum) > 0)

In [13]:
train_df.subset_present.value_counts(normalize=True)

subset_present
False    0.967838
True     0.032162
Name: proportion, dtype: float64

##### 3 % is highly imbalanced set, lets try subsampling

In [115]:
sp = train_df.loc[train_df.subset_present == False].sample(frac=0.15, random_state=121)

subsampled_train = pd.concat([sp, train_df.loc[train_df.subset_present == True]], ignore_index=True)

subsampled_train.subset_present.value_counts(normalize=True)

subset_present
False    0.818641
True     0.181359
Name: proportion, dtype: float64

In [10]:
subsampled_train["ner_tags_subset"] = subsampled_train["ner_tags_subset"].apply(lambda x: [ner_tags[y] for y in x])

NameError: name 'subsampled_train' is not defined

##### expanding tags for xx.xx types

In [12]:
import re
def expand_tags(tokenizer, token_list, tag_list):
    
    final_tags = []
    
    counter = 0
    
    for t, l in zip(token_list, tag_list):
        
        t = re.sub(r'[^\x00-\x7F]+', '', t)
        if t == "":
            continue
        counter +=1
        
        
#         print(counter, l, t, counter)

        temp_counts = 0
        
        for toks in tokenizer.tokenize(t):
#             print(t, toks)
            
            
#             print(toks, temp_counts)
            if toks.startswith("##"):
                continue
            else:
                temp_counts +=1
                if temp_counts > 1:
                    if l != "O":
                        l = l.replace("B-", "I-")
#                 print(counter, "append", l, counter)
                    
                final_tags.append(l)           
    return final_tags

def expand_toks(tokenizer, token_list):
    
    final_toks = []
    for i, t in enumerate(token_list):
        t = re.sub(r'[^\x00-\x7F]+', '', t)
        if t == "":
            continue
        
        final_toks += combine_subtoks(tokenizer.tokenize(t))
                         
    return final_toks


def combine_subtoks(toks):
    
    comb =[]
    
    curr = []
    for i,t in enumerate(toks):
#         print(i, curr)
        
        if t.startswith("##"):
            t = t[2:]
            curr.append(t)
        else:
            if len(curr) > 0:
                comb.append("".join(curr))
            curr= [t]
#             if i == len(toks)-1:
#                 print(i, comb)
    comb.append("".join(curr))
                
            
            
    return comb

from transformers import AutoTokenizer
import time

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [166]:
subsampled_train.shape

(159672, 11)

In [167]:
subsampled_train["tok_len"] = subsampled_train["tokens"].apply(lambda x: len(x))

subsampled_train["toks_exp_len"] = subsampled_train["toks_exp"].apply(lambda x: len(x))

subsampled_train["tags_len"] = subsampled_train["ner_tags_subset"].apply(lambda x: len(x))

subsampled_train["tags_exp_len"] = subsampled_train["ner_expanded"].apply(lambda x: len(x))

In [168]:
subsampled_train.loc[subsampled_train.toks_exp_len != subsampled_train.tags_exp_len]#.ner_tags_subset.iloc[0]

,id,tokens,ner_tags,ner_tags_subset,subset_present,toks_exp,ner_expanded,tok_len,toks_exp_len,tags_len,tags_exp_len


In [170]:
subsampled_train["ner_expanded"].explode().value_counts()

ner_expanded
O                                                 7545947
I-DebtInstrumentInterestRateStatedPercentage        28842
I-DebtInstrumentBasisSpreadOnVariableRate1          20898
I-LineOfCreditFacilityMaximumBorrowingCapacity      17488
I-DebtInstrumentFaceAmount                          15441
B-DebtInstrumentInterestRateStatedPercentage        15208
B-LineOfCreditFacilityMaximumBorrowingCapacity      11693
B-DebtInstrumentBasisSpreadOnVariableRate1          11445
B-DebtInstrumentFaceAmount                          10749
I-DebtInstrumentMaturityDate                         4320
B-DebtInstrumentMaturityDate                         1440
Name: count, dtype: int64

In [173]:
i = 142165
[(x,y) for x,y in zip(subsampled_train.loc[i,"ner_tags_subset"], subsampled_train.loc[i,"tokens"]) if x!="O"]

[('B-DebtInstrumentBasisSpreadOnVariableRate1', '0.375'),
 ('B-DebtInstrumentBasisSpreadOnVariableRate1', '1.375')]

In [175]:
# i = 139011
[(x,y) for x,y in zip(subsampled_train.loc[i,"ner_expanded"], subsampled_train.loc[i,"toks_exp"]) if x!="O"]

[('B-DebtInstrumentBasisSpreadOnVariableRate1', '0'),
 ('I-DebtInstrumentBasisSpreadOnVariableRate1', '.'),
 ('I-DebtInstrumentBasisSpreadOnVariableRate1', '375'),
 ('B-DebtInstrumentBasisSpreadOnVariableRate1', '1'),
 ('I-DebtInstrumentBasisSpreadOnVariableRate1', '.'),
 ('I-DebtInstrumentBasisSpreadOnVariableRate1', '375')]

In [177]:
train_df["ner_tags_subset"] = train_df["ner_tags_subset"].apply(lambda x: [ner_tags[y] for y in x])

In [15]:
valid_df["ner_tags_subset"] = valid_df["ner_tags_subset"].apply(lambda x: [ner_tags[y] for y in x])

In [21]:
st = time.time()
valid_df["toks_exp"] = valid_df["tokens"].apply(lambda x: expand_toks(tokenizer, x))

et = time.time()

print("s", et-st)

valid_df["ner_expanded"] = valid_df.apply(lambda x: expand_tags(
    tokenizer, x["tokens"], x["ner_tags_subset"]), axis=1)

et2=time.time()

print("s", et2-st)

s 227.0125434398651
s 448.4984321594238


In [22]:
valid_df["ner_tags_subset"].explode().value_counts()

ner_tags_subset
O                                                 5187081
B-DebtInstrumentInterestRateStatedPercentage         1841
B-LineOfCreditFacilityMaximumBorrowingCapacity       1692
B-DebtInstrumentBasisSpreadOnVariableRate1           1684
B-DebtInstrumentFaceAmount                           1346
I-DebtInstrumentMaturityDate                          507
B-DebtInstrumentMaturityDate                          169
Name: count, dtype: int64

In [23]:
valid_df["ner_expanded"].explode().value_counts()

ner_expanded
O                                                 5282393
I-DebtInstrumentInterestRateStatedPercentage         3544
I-DebtInstrumentBasisSpreadOnVariableRate1           3090
I-LineOfCreditFacilityMaximumBorrowingCapacity       2542
I-DebtInstrumentFaceAmount                           1976
B-DebtInstrumentInterestRateStatedPercentage         1841
B-LineOfCreditFacilityMaximumBorrowingCapacity       1692
B-DebtInstrumentBasisSpreadOnVariableRate1           1684
B-DebtInstrumentFaceAmount                           1346
I-DebtInstrumentMaturityDate                          507
B-DebtInstrumentMaturityDate                          169
Name: count, dtype: int64

In [24]:
valid_df.shape

(112494, 6)

In [180]:
subsampled_train[["id", "toks_exp", "ner_expanded"]].sample(frac=1).to_csv("data/subsampled_train.csv", index=False)

In [19]:
train_df[["id", "tokens", "ner_tags_subset"]].sample(frac=1).to_csv("data/train_proc.csv", index=False)

In [28]:
valid_df[["id", "toks_exp", "ner_expanded"]].to_csv("data/valid_proc.csv", index=False)

In [25]:
train_df.isna().sum()

id                 0
tokens             0
ner_tags           0
ner_tags_subset    0
subset_present     0
dtype: int64

In [25]:
# verify if tokens are aligned
valid_df["tok_len"] = valid_df["tokens"].apply(lambda x: len(x))

valid_df["toks_exp_len"] = valid_df["toks_exp"].apply(lambda x: len(x))

valid_df["tags_len"] = valid_df["ner_tags_subset"].apply(lambda x: len(x))

valid_df["tags_exp_len"] = valid_df["ner_expanded"].apply(lambda x: len(x))

In [26]:
valid_df.loc[valid_df.toks_exp_len != valid_df.tags_exp_len]#.ner_tags_subset.iloc[0]

,id,tokens,ner_tags,ner_tags_subset,toks_exp,ner_expanded,tok_len,toks_exp_len,tags_len,tags_exp_len


In [27]:
valid_df[["id", "toks_exp", "ner_expanded"]].to_csv("data/valid_proc.csv", index=False)